## Load/import packages

In [1]:
import json
import time
import scipy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import kerastuner as kt

from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.utils import class_weight

from os import listdir
from os.path import join, splitext, normpath

# Import modules to run custom FW-RNN cell
from tensorflow.python.keras.layers.recurrent import (
    _generate_zero_filled_state_for_cell,
    _generate_zero_filled_state,
    ops,
    tensor_shape,
    activations,
    initializers,
    regularizers,
    nest,
    array_ops,
)

# Import variables and functions from my own scripts
from functions import plot_history, arr_replacevalue
from load_features import (
    train_features_AW2,
    val_features_AW2,
    train_labels_AW2,
    val_labels_AW2,
    labels_reshaper,
    features_reshaper,
)

%matplotlib inline

# Limit GPU memory usage
for device in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(device, True)

# Prepare data

In [2]:
# Reshape data to specified sequence length
length = 60
seq_train_features = features_reshaper(train_features_AW2, length) # divisible 13, 39, 197
seq_val_features = features_reshaper(val_features_AW2, length)

seq_train_labels = labels_reshaper(train_labels_AW2, length)
seq_val_labels = labels_reshaper(val_labels_AW2, length)

In [3]:
def comp_sampleweights(labels):
    # Convert one-hot encoded labels back to label integers
    train_label_ints = np.argmax(labels, axis=2)

    # Compute class weights with sklearn
    class_weights = class_weight.compute_class_weight(
        "balanced", np.unique(train_label_ints), train_label_ints.flatten()
    )
    d_class_weights = dict(enumerate(class_weights))

    # Pass a 2D array with shape (samples, sequence_length), to apply a different weight to every timestep of every sample
    return arr_replacevalue(train_label_ints, d_class_weights)
    
train_samples_weights = comp_sampleweights(seq_train_labels) 

C:\Users\ion\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4 5 6], y=[0 0 0 ... 6 6 6] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


# Build model

In [12]:
# Build model with sequential api
def build_model(activation_function):
    model = Sequential()
    model.add(tf.keras.Input(shape=(seq_train_features.shape[1], seq_train_features.shape[2])))
    model.add(layers.SimpleRNN(128))
    model.add(layers.Dense(7, activation="softmax", name="Dense_Output"))
    model.compile(
        optimizer="adagrad",
        loss=CategoricalCrossentropy(label_smoothing=0.1),
        metrics=["accuracy"],
        run_eagerly=False,
    )
    return model


# Train + Evaluate model

In [13]:
# from sklearn.model_selection import KFold
# for train, test in kfold.split()

In [ ]:
for act in ['relu', 'tanh']:
    fw_rnn = build_model(batchsize, act)
    fw_rnn.summary()

    # Access tensorboard in cmd of the main repo folder with following code:
    # tensorboard --logdir='logs/'
    name = 'FW-RNN_1layer64_rmsprop_identitymatrix0.05_{}_{}'.format(act, int(time.time()))
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/{}'.format(name))

    # Set callbacks for model training
#     csvlog = tf.keras.callbacks.CSVLogger(
#         "data/models/FW-RNN_performance.csv", separator=",", append=False
#     )

    history_best = fw_rnn.fit(
        seq_train_features[:a],
        seq_train_labels[:a],
        batch_size=batchsize,
        sample_weight=train_samples_weights[:a],
        validation_data=(
            seq_val_features[:b],
            seq_val_labels[:b],
        ),
        callbacks=[
#                 csvlog, 
#                 tb_callback
        ],
        epochs=150,
        verbose=2,
        shuffle=True,
    )